In [1]:
#%%time
#create DASK scheduler and worker remote containers
#this will take at least one minute as there are delays added on purpose to allow containers to spawn
#on successful run you should see scheduler URL printed

#!python3 daskmaster.py

In [2]:
# defined as global
daskschurl = ""

In [3]:
import cdsw
import os
import time

def dask_distributed_launch(nworkers=1,ncpu=2,nmemory=4):
    # modify global copy
    global daskschurl
    
    #check if already running
    if daskschurl!="":
        #print(cdsw.list_workers())
        print(" Dask Scheduler Already Launched " + daskschurl)
        return(daskschurl)
    
    # Launch CDSW workers. These are engines that will run in 
    # the same project, execute a given code or script, and exit.
    # Scheduler engine will keep running in background until session is closed
    dask_scheduler = cdsw.launch_workers(n=1, cpu=2, memory=4, 
                                  kernel="python3",script="daskschedular.py")


    # IP of launched container comes up unknown for a while
    # Wait for a while so IP is available in data structure
    time.sleep(30)

    # Get schedular IP
    schedulerid = dask_scheduler[0]["id"]
    listtemp = cdsw.list_workers()

    for x in listtemp:
      if x["id"] == schedulerid:
        schedulerip = x["ip_address"]


    print(" Scheduler IP: " + schedulerip)

    #Scheduler protocol and port - defaults from Dask
    schproto = "tcp://"
    schport = ":8786"

    schloc = schproto + schedulerip + schport
    print(" Scheduler URL: " + schloc)

    dask_client = []
    # Launch at least one Dask Worker
    for c in range(nworkers):
        dask_client = dask_client + cdsw.launch_workers(n=1, cpu=ncpu, memory=nmemory, 
                                  kernel="python3",script="daskworker.py",
                                      env={"DASKSCHURL": schloc})

        # wait for a while until the container is launched successfully
        time.sleep(10)
    
    #set scheduler URL as environment variable
    #os.putenv("DASKSCHURL", schloc)
    daskschurl = schloc

    #return scheduler URL
    return(schloc)


def dask_stop_workers():
    global daskschurl
    cdsw.stop_workers()
    daskschurl = ""
    

In [4]:
def dask_test():
    from dask.distributed import Client
    client = Client(daskschurl)
    import dask.array as da
    x = da.random.random((40000,40000),chunks=(1000,1000))
    y = da.exp(x).sum()
    print(" Result of DASK distributed array test: " + str(y.compute()) + "\n")


In [5]:
#stop any previous dask distributed containers
dask_stop_workers()

In [6]:
#launch scheduler and worker container(s) - you can specify number of workers as argument
#to relaunch - first call dask_stop_workers() and then call this again
dask_distributed_launch(2,4,8)

 Scheduler IP: 10.10.25.197
 Scheduler URL: tcp://10.10.25.197:8786


'tcp://10.10.25.197:8786'

In [7]:
#check if global variable has the right URL
#we will use this to register a client
print(daskschurl)

tcp://10.10.25.197:8786


In [10]:
import pandas as pd
import cdsw
#any previously stopped pods may show up with status failed
#we are not showing those
workers_list = cdsw.list_workers()
print(" === List of launched running pods === ")
print(" ===      (scheduler + workers)    === ")

#collect relevant fields
workersl = []
for l in workers_list:
    #print(l)
    #print(" id: " + l["id"] + " IP Addr: " + l["ip_address"] + " CPUs: " + str(l["cpu"]) + " Memory: " + str(l["memory"]) \
    #     + " Status: " + l["status"])
    workersl = workersl + [[l["id"],l["ip_address"],l["cpu"],l["memory"],l["status"]]]

workersactive = pd.DataFrame(workersl,columns=["Id","IP Address","CPUs","Memory","Status"])
print(workersactive[workersactive.Status=="running"])

 === List of launched running pods === 
 ===      (scheduler + workers)    === 
                 Id    IP Address  CPUs  Memory   Status
0  eu3vmaudzjnfw0h1   10.10.21.92     4       8  running
1  uls2ad95u6oxi2oe     10.10.8.3     4       8  running
2  c7xg96ixga8le7l2  10.10.25.197     2       4  running


In [11]:
%%time
#check distributed dask is working
dask_test()

#Register a DASK client and run a test
#from dask.distributed import Client
#client = Client(daskschurl)
#import dask.array as da
#x = da.random.random((40000,40000),chunks=(1000,1000))
#y = da.exp(x).sum()
#print("DASK test result: ") 
#print(y.compute())

 Result of DASK distributed array test: 2749264832.62

CPU times: user 6.07 s, sys: 25.7 ms, total: 6.1 s
Wall time: 11.5 s


In [9]:
%%time
#Read file from S3
#configure access via aws configure in terminal
#need pip3 install awscli for it to work

#Dask dataframe has poor ability to infer column types
#Need to be fixed manually
#options can be modified in ~/.config/dask/distributed.yaml 


import dask.dataframe as dd
import s3fs
df = dd.read_csv("s3://harshalpatil-s3/loans_accepted_2007_to_2018Q4.csv", \
                blocksize="25MB",sample=10000000,dtype={'id': 'object', \
       'sec_app_earliest_cr_line': 'object', 'desc': 'object'})

CPU times: user 801 ms, sys: 171 ms, total: 972 ms
Wall time: 12.5 s


In [10]:
#check number of partitions 

#df.known_divisions
#df.set_index("grade")
#df = df.repartition(npartitions=10000)
df.npartitions


68

In [11]:
#Drop any columns with ALL missing values - else may cause certain dask df functions to fail
nonnullcnt=pd.DataFrame(df.count().compute())

/usr/local/lib/python3.6/dist-packages/dask/core.py:118: DtypeWarning: Columns (129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
/usr/local/lib/python3.6/dist-packages/dask/core.py:118: DtypeWarning: Columns (145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
/usr/local/lib/python3.6/dist-packages/dask/core.py:118: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
/usr/local/lib/python3.6/dist-packages/dask/core.py:118: DtypeWarning: Columns (129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
/usr/local/lib/python3.6/dist-packages/dask/core.py:118: DtypeWarning: Columns (59,129,130,131,13

In [12]:
from IPython.display import display, HTML
display(HTML(nonnullcnt.to_html()))

,0
id,2260701
member_id,0
loan_amnt,2260668
funded_amnt,2260668
funded_amnt_inv,2260668
term,2260668
int_rate,2260668
installment,2260668
grade,2260668
sub_grade,2260668


In [15]:
####
display(HTML(df.head(1).to_html()))

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df=df.drop(columns=["member_id"])

In [33]:
summarydf=df.describe().compute()

In [16]:
from sklearn.datasets import load_wine
data = load_wine()
#since the data is numpy series we will not use df = dd.from_pandas(data[‘data’])
#dask has various ways to convert numpy and pandas to dask dataframes      
df = dd.from_array(data['data'])
df.columns = data['feature_names']

df.describe().compute()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [12]:
#Use the wine dataset
#from sklearn.datasets import load_wine
#data = load_wine()

from dask.distributed import Client
client = Client(daskschurl)

from dask import dataframe as dd
f = dd.read_csv("datagenclassf.csv")
df=f.drop(columns=["target"])
#since the data is numpy series we will not use df = dd.from_pandas(data[‘data’])
#dask has various ways to convert numpy and pandas to dask dataframes      
#df = dd.from_array(data['data'])
#df.columns = data['feature_names']

#print a few lines
print("\n Dataframe: ")
print(df.head())

#Get target variable
dt=f["target"]
#dt = dd.from_array(data['target'])
#dt.columns = ["target"]
      
#print target classes example
print("\n Target: ")
print(dt.head())      

# train and test split
from dask_ml.model_selection import train_test_split
train, test, train_labels, test_labels = train_test_split(df,dt,random_state=123)      
      
#xgboost
from dask_ml.xgboost import XGBClassifier
est = XGBClassifier()      
      
#fit model      
model = est.fit(train, train_labels)



 Dataframe: 
         V1        V2        V3        V4        V5        V6        V7  \
0  0.083939 -0.487069  0.299579 -1.058957  0.889796 -2.263859 -2.070858   
1  2.652801 -0.196639  2.365537  2.485281 -0.729860 -0.733576  1.608655   
2 -1.385589 -0.478161  1.173640  0.186778 -0.776586 -0.836241 -1.902210   
3  1.045551  2.216028 -2.354473 -2.822850  0.565806  1.694350  1.701352   
4 -0.160565  0.355196  2.353566  1.700229  1.504608 -3.362701 -2.658327   

         V8        V9       V10       V11       V12       V13       V14  \
0 -0.880715 -0.151838  2.742516  5.503376  0.132226 -0.338907 -2.415953   
1  1.118913 -0.239675 -0.298033 -0.879184  1.156604 -0.517310  3.773901   
2  0.898840  0.628935 -1.163406  1.810582  1.514793  2.645236 -0.593265   
3 -2.089311  1.700781 -1.690056  1.317296  2.174892 -0.954333  2.303896   
4 -1.874578 -2.123515  0.561999  5.819872 -0.258777  0.373247 -1.623749   

        V15  
0 -1.292555  
1 -0.591695  
2 -0.629657  
3  1.243514  
4 -0.163059  


In [13]:
#which features contribute most
import pandas as pd
featureimp = pd.DataFrame(model.feature_importances_)
featureimp.columns = ['classifier_feature_importance']
featureimp["variable"] = df.columns
print("\n\n === Xgboost Classifier Feature Importance: === ")
print(featureimp.sort_values(by="classifier_feature_importance", ascending=False))
#featureimp.to_csv()


#predictions
ypred = model.predict(test)

#sample some predictions
print("\n Sample initial five predictions: ")      
print(ypred[[0,1,2,3,4]].compute())

#ensure model is predicting all classes - not just 0
print("\n Check classes other than zero predicted: ")
print(ypred[ypred>0].compute())
      
#check accuracy on test set      
from dask_ml import metrics
print("\n\n Model Accuracy: ")      
print(metrics.accuracy_score(test_labels,model.predict(test)))
      
print("\n === End Dask Xgboost === \n")




 === Xgboost Classifier Feature Importance: === 
    classifier_feature_importance variable
7                        0.219154       V8
13                       0.108420      V14
6                        0.090464       V7
11                       0.088831      V12
8                        0.086293       V9
9                        0.071998      V10
2                        0.070237       V3
4                        0.057946       V5
10                       0.051210      V11
0                        0.045434       V1
12                       0.037099      V13
1                        0.034969       V2
3                        0.023079       V4
5                        0.014867       V6
14                       0.000000      V15

 Sample initial five predictions: 
[1 1 1 1 1]

 Check classes other than zero predicted: 
[1 1 1 ..., 1 1 1]


 Model Accuracy: 
0.500877149115

 === End Dask Xgboost === 

